In [1]:
import re
import string
import os
import sys
import pandas as pd

# Requires eGenix.com mx Base Distribution
# http://www.egenix.com/products/python/mxBase/
try:
    from mx.DateTime import *
except ImportError:
    print """
Requires eGenix.com mx Base Distribution
http://www.egenix.com/products/python/mxBase/"""

# Predefined strings.
numbers = "(^a(?=\s)|one|two|three|four|five|six|seven|eight|nine|ten| \
          eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen| \
          eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty| \
          ninety|hundred|thousand)"
day = "(monday|tuesday|wednesday|thursday|friday|saturday|sunday)"
week_day = "(monday|tuesday|wednesday|thursday|friday|saturday|sunday)"
month = "(january|february|march|april|may|june|july|august|september| \
          october|november|december)"
dmy = "(year|day|week|month)"
rel_day = "(today|yesterday|tomorrow|tonight|tonite|fortnight)"
exp1 = "(before|after|earlier|later|ago|back)"
exp2 = "(this|next|last|of|till|to)"
exp3 = "(from|since|between|with)"
exp4 = "(treatment|therapy)"
exp5 = "(start)"
iso = "\d+[/-]\d+[/-]\d+ \d+:\d+:\d+\.\d+"
year = "((?<=\s)\d{4}|^\d{4})"
drug = "(Pembrolizumab|Nivolumab|Ipilimumab)"

regxp1 = "((\d+|(" + numbers + "[-\s]?)+) " + dmy + "s? " + "(" + exp1 + "|" + exp3 +"))"
regxp2 = "(" + "("+ exp3 + "|" + exp2 + ")" + numbers + " (" + dmy + "|" + week_day + "|" + month +"))"
regxp3 = "(" +drug +")"
regxp4 =  "(" + exp4 +  ")"
regxp5 = "("+exp5 + "(" + "ed?" +"|"+"ing?" + ")" + "|" + exp3 +")"
reg1 = re.compile(regxp1, re.IGNORECASE)
reg2 = re.compile(regxp2, re.IGNORECASE)
reg3 = re.compile(rel_day, re.IGNORECASE)
reg4 = re.compile(iso)
reg5 = re.compile(year)
reg6 = re.compile(regxp3,re.IGNORECASE)
reg7 = re.compile(regxp4,re.IGNORECASE)
reg8 = re.compile(regxp5,re.IGNORECASE)
def tag(text):

    # Initialization
    timex_found = []

    # re.findall() finds all the substring matches, keep only the full
    # matching string. Captures expressions such as 'number of days' ago, etc.
    found = reg1.findall(text)
    found = [a[0] for a in found if len(a) > 1]
    for timex in found:
        timex_found.append(timex)

    # Variations of this thursday, next year, etc
    found = reg2.findall(text)
    found = [a[0] for a in found if len(a) > 1]
    for timex in found:
        timex_found.append(timex)

    # today, tomorrow, etc
    found = reg3.findall(text)
    for timex in found:
        timex_found.append(timex)

    # ISO
    found = reg4.findall(text)
    for timex in found:
        timex_found.append(timex)

    # Year
    found = reg5.findall(text)
    for timex in found:
        timex_found.append(timex)
        
    found = reg6.findall(text)
    found = [a[0] for a in found if len(a) > 1]
    for timex in found:
        timex_found.append(timex)
        
    found = reg7.findall(text)
    found = [a[0] for a in found if len(a) > 1]
    for timex in found:
        timex_found.append(timex)
        
    found = reg8.findall(text)
    found = [a[0] for a in found if len(a) > 1]
    for timex in found:
        timex_found.append(timex)
       
    

    # Tag only temporal expressions which haven't been tagged.
    for timex in timex_found:
        text = re.sub(timex + '(?!</TIMEX2>)', '<TIMEX2>' + timex + '</TIMEX2>', text)

    return text





Requires eGenix.com mx Base Distribution
http://www.egenix.com/products/python/mxBase/


In [7]:
x = pd.read_csv('ULF_100_selected.csv')

x.head(10)

Unnamed: 0     id       User  \
0        2025  21740  Bubbles -   
1        2026  18822  Bubbles -   
2        2027  12472  Bubbles -   
3        2028  10209  Bubbles -   
4        2029  22141  Bubbles -   
5        2030   1698  Bubbles -   
6        2031  13665  Bubbles -   
7        2032  25555  Bubbles -   
8        2033  12743  Bubbles -   
9        2034  22714  Bubbles -   

                                 Tokenized.Sentences             date  Tag  \
0  '\'Hi Betsy,   I took Nivolumab Nivolumab the ...    5/9/2017 0:00    1   
1  "'I really do understand your conundrum...havi...    9/2/2016 0:00    1   
2  '"\\xc3\\x82\\xc2  Have you tried Ipilimumab N...  11/11/2016 0:00    1   
3  "'\\xc3\\x82\\xc2  Meaning....immunotherapy (l...  11/11/2016 0:00    1   
4  "'\\xc3\\x82\\xc2 We now have Ipilimumab BRAF ...   4/20/2017 0:00    1   
5  "'\\xc3\\x82\\xc2  Here is a post I made when ...    5/9/2015 0:00    1   
6  '\'\\xc3\\x82\\xc2 Here is a report that demon...    3/3/2017 0:00    1   
7  "'Ipilimumab has a response rate of about 15% ...  11/22/2016 0:00    1   
8  "'\\xc3\\x82\\xc2  Two studies with BRAFi as a...   1/10/2016 0:00    1   
9  "'\\xc3\\x82\\xc2  Immunotherapy: Ipilimumab I...    6/3/2015 0:00    1   

   forum  Count  
0      1    524  
1      1    524  
2      1    524  
3      1    524  
4      1    524  
5      1    524  
6      1    524  
7      1    524  
8      1    524  
9      1    524

In [5]:
z = x['Tokenized.Sentences']


In [15]:
A =[] 
for i in z:
    a = "".join(str(x) for x in i)
    A.append(a) 


In [18]:
from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk.tokenize import sent_tokenize
from nltk import pos_tag


In [20]:
#words_1= preprocessing(x['Tokenized.Sentences'])

In [33]:
test = "this Nivolumab last 1 week"
print tag(test)

x= tag(test)

this <TIMEX2>Nivolumab</TIMEX2> last 1 week


In [181]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

def find_between_r( s, first, last ):
    try:
        start = s.rindex( first ) + len( first )
        end = s.rindex( last, start )
        return s[start:end]
    except ValueError:
        return ""




In [68]:
print find_between( x, "<TIMEX2>", "</TIMEX2>" )
print find_between_r( x, "<TIMEX2>", "</TIMEX2>" )

Nivolumab
25 weeks back


In [58]:
A =[] 
for i in z:
    a = "".join(str(x) for x in i)
    print tag(a)

list_1 = tag(a)

'\'Hi Betsy,   I took Nivolumab Nivolumab the "other" anti-PD-1 product with basically identical side effects and response rates as Pembrolizumab Pembrolizumab for 2 1/<TIMEX2>2 years from</TIMEX2> Dec <TIMEX2>2010</TIMEX2> - June of <TIMEX2>2013</TIMEX2>.\''
"'I really do understand your conundrum...having been myself Stage III from <TIMEX2>2003</TIMEX2> to <TIMEX2>2010</TIMEX2> with no options other than srugical removal and Interferon I did NOT take Interferon and despite progressing, I do not regret it.'"
'"\\xc3\\x82\\xc2  Have you tried Ipilimumab Nivolumab I\'ll take the trials you mentioned from the bottom up and tell you what I have:   To Brian\'s comment."', "'The other part about TIL is how many ways they are working on doing it currently.'"
"'\\xc3\\x82\\xc2  Meaning....immunotherapy (like the Ipilimumab Nivolumab combo) with SRS or gamma knife to brain mets...and NOW....doing well!!'"
"'\\xc3\\x82\\xc2 We now have Ipilimumab BRAF inhibitors and two anti-PD1 products that c

In [18]:
x_1 = pd.read_csv('New_TE.csv')

In [2]:
user_100 = pd.read_csv('User_Level_Selected_100.csv')



In [3]:
user_100.head(5)

Unnamed: 0     id       User  \
0        2026   4711  Bubbles -   
1        2027  15646  Bubbles -   
2        2028  10924  Bubbles -   
3        2029   6190  Bubbles -   
4        2030  22372  Bubbles -   

                                 Tokenized.Sentences             date  Tag  \
0  'Sorry for all that you and your husband have ...  11/11/2016 0:00    1   
1  "Merck's anti-PD1, Pembrolizumab now called Pe...  12/22/2014 0:00    1   
2  'Yes, Cooper.', '\xc3\x82\xc2  Press is one th...  12/22/2014 0:00    1   
3  'You are very kind...but...', "\xc3\x82\xc2 \x...  10/19/2016 0:00    1   
4  "Hey Steve,   I've not heard of an Ipilimumab ...  10/31/2014 0:00    1   

   forum  Count  
0      1    524  
1      1    524  
2      1    524  
3      1    524  
4      1    524

In [31]:
user_ = user_100.groupby(['id','User',toke])['date'].agg(['min'])
#user_ = user_100.grousort_values('User')

In [32]:
user_.head(5)

min
id User                              
2  BrianP -            8/14/2013 0:00
10 FormerCaregiver -  10/27/2011 0:00
16 jmmm -               4/3/2013 0:00
19 rick1981 -          1/12/2015 0:00
24 Tim--MRF -          5/14/2012 0:00

In [26]:
A_list_ = pd.DataFrame(x_1['Tokenized.Sentences'].map(lambda x:tag(x)))
    
    
    

In [127]:
Timex_tag = A_list['Tokenized.Sentences'].map(lambda x:find_between( x, "<TIMEX2>", "</TIMEX2>" ))
#Timex_tag_1 = A_list['Tokenized.Sentences'].map(lambda x:find_between_r( x, "<TIMEX2>", "</TIMEX2>" ))

In [171]:
from  BeautifulSoup import BeautifulSoup 


In [242]:
s = 'this <TIMEX2>Nivolumab</TIMEX2>since <TIMEX2>25 weeks back</TIMEX2>'
#s = 'this since <TIMEX2>25 weeks back</TIMEX2>'


In [248]:
soup = BeautifulSoup(s)

type(soup)

BeautifulSoup.BeautifulSoup

In [175]:
row = A_list['Tokenized.Sentences'].map(lambda x:find_between(x,"<TIMEX2>", "</TIMEX2>"))

TypeError: expected string or buffer

In [173]:
type(soup)

BeautifulSoup.BeautifulSoup

In [217]:
m = re.compile('<TIMEX2>(.*?)(/TIMEX2)').search()
print m.find()

AttributeError: '_sre.SRE_Match' object has no attribute 'find'

In [20]:
Temporal_words = pd.DataFrame(A_list_['Tokenized.Sentences'].map(lambda x:re.findall('<TIMEX2>(.*?)</TIMEX2>',x)))



In [21]:
x_1['temp']= Temporal_words



In [22]:
x_1.to_csv('TE_new.csv')

In [44]:
x.to_csv('temporal_words_data_10.csv')

In [188]:
import re
for tag in soup.find_all(re.compile(regxp1, re.IGNORECASE)):
    print(tag.name)

TypeError: 'NoneType' object is not callable

In [60]:
split_ulf = pd.read_csv("Split_ULF.csv")

split_1 = split_ulf[:1615]
split_2 = split_ulf[1616:3230]
split_3 = split_ulf[3231:4845]




In [62]:
split_1.to_csv('split_1.csv')
split_2.to_csv('split_2.csv')
split_3.to_csv('split_3.csv')

In [ ]:
r"""(?:^(?:never|no|nothing|nowhere|noone|none|not|doesn't|haven't
            havent|hasnt|hadnt|cant|couldnt|shouldnt|rather than|
            wont|wouldnt|dont|doesnt|didnt|isnt|arent|aint|instead)$)|n't"""

In [30]:
NEGATION = r"""(?:^(?:never|no|nothing|nowhere|noone|none|not|doesn't|haven't|
            havent|hasnt|hadnt|cant|couldnt|shouldnt|rather than|
            wont|wouldnt|dont|doesnt|didnt|isnt|arent|aint|instead)$)|n't"""
NEGATION_RE = re.compile(NEGATION, re.VERBOSE)

SyntaxError: invalid syntax (<ipython-input-30-f22b2031d908>, line 1)

In [26]:
text = "Also, Ipilimumab doesn't seem to be an option, since we??? are not eligible for compassionate use."

In [34]:
def neg_tag(text):

    # Initialization
    neg_found = []
    orig_text = text

    ##WITH PRIORITY

    # Variations of this thursday, next year, etc
    found = NEGATION_RE.findall(text)
    found = [a[0] for a in found if len(a) > 1]
    for neg in found:
        neg_found.append(neg)
        text = re.sub(neg,'',text)
    
    
    for neg in neg_found:
        orig_text = re.sub( neg +'(?!</NEG>)', '<NEG>' + neg + '</NEG>', orig_text)

    return orig_text

In [35]:
text_neg = neg_tag(text)

In [36]:
text_neg

"Also, Ipilimumab does<NEG>n</NEG>'t seem to be a<NEG>n</NEG> optio<NEG>n</NEG>, si<NEG>n</NEG>ce we??? are <NEG>n</NEG>ot eligible for compassio<NEG>n</NEG>ate use."

In [37]:
tagged_user = pd.read_csv('100_users_tagged.csv')

In [50]:
import re
import string
import os
import sys
import pandas as pd
import numpy as np

# NEGATION = r"""
#     (?:
#         ^(?:never|no|nothing|nowhere|noone|none|not|
#             havent|hasnt|hadnt|cant|couldnt|shouldnt|
#             wont|wouldnt|dont|doesnt|didnt|isnt|arent|aint
#         )$
#     )
#     |
#     n't"""

NEGATION = "(never|\bno\b|over|against|nothing|denied|nowhere|none|not|wasn't|wasnt|havent|haven't|hasnt|hasn't|hadnt|hadn't|cant|can't|couldnt|couldn't|shouldnt|shouldn't|wont|won't|wouldnt|would not|wouldn't|dont|don't|doesnt|dosen't|didnt|didn't|isnt|isn't|arent|aren't|aint|n't|instead|but|rather)"
NEGATION_RE = re.compile(NEGATION, re.VERBOSE)

def neg_tag(text):

    # Initialization
    neg_found = []
    orig_text = text

    ##WITH PRIORITY

    # Variations of this thursday, next year, etc
    found = NEGATION_RE.findall(text)
    # found = [a[0] for a in found if len(a) > 1]
    for neg in found:
        neg_found.append(neg)
        text = re.sub(neg,'',text)
    
    print found
    for neg in neg_found:
        orig_text = re.sub( neg +'(?!</NEG>)', '<NEG>' + neg + '</NEG>', orig_text)

    return orig_text


In [53]:
N_list_ = pd.DataFrame(tagged_user['Tokenized.Sentences'].map(lambda x:neg_tag(x)))

Neg_mentions = pd.DataFrame(N_list_['Tokenized.Sentences'].map(lambda x:np.unique(re.findall('<NEG>(.*?)</NEG>',x))))


tagged_user['Negations'] = Neg_mentions

[]
[]
['not']
[]
[]
[]
['but', 'not']
[]
[]
['but']
['but']
['but', 'not']
[]
['rather']
[]
[]
[]
[]
[]
['but', 'never']
[]
[]
[]
[]
[]
['but']
[]
['over']
[]
[]
['over']
[]
["didn't"]
['over']
["didn't"]
[]
['but']
["n't"]
[]
[]
[]
[]
[]
[]
[]
[]
['over']
[]
[]
[]
[]
['over']
['nothing']
["haven't"]
[]
[]
[]
['not']
[]
[]
[]
['but']
[]
[]
[]
[]
[]
[]
[]
['nothing']
[]
[]
[]
['but']
[]
['over']
[]
[]
[]
[]
[]
['but']
[]
['nothing']
[]
[]
[]
[]
[]
[]
[]
[]
[]
["don't"]
['not', "n't"]
['not']
[]
[]
['over']
['cant', 'over']
[]
[]
[]
[]
["wouldn't"]
[]
[]
[]
[]
[]
["didn't"]
[]
['over']
[]
['but', "don't"]
[]
['over']
[]
[]
['cant', 'cant']
['but']
[]
['over']
['over']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['over', 'over']
[]
["don't", 'but']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['over', 'but']
[]
[]
[]
[]
['couldnt', 'but', 'not', 'dont', 'but', 'not']
['not', 'not', 'but', 'dont']
['but', 'wouldnt', 'dont', 'but']
[]
['nothing', 'couldnt', 'but', 'not', 'but', 'but']
['but', 'not', 'not', 

In [54]:
tagged_user.to_csv("tagged_user.csv")

In [51]:
selected_users = pd.read_csv('selected_final_100_users_tagged.csv')

In [52]:
N_list_1 = pd.DataFrame(selected_users['Tokenized.Sentences'].map(lambda x:neg_tag(x)))
Neg_mentions = pd.DataFrame(selected_users['Tokenized.Sentences'].map(lambda x:np.unique(re.findall('<NEG>(.*?)</NEG>',x))))
selected_users['Negations'] = Neg_mentions


[]
[]
[]
[]
[]
[]
[]
['not', 'but', 'over']
[]
[]
[]
[]
['over']
[]
[]
[]
[]
[]
['not']
['over']
[]
[]
[]
[]
[]
[]
['not']
['but']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['but']
[]
[]
[]
[]
[]
[]
[]
[]
['but']
['not']
[]
['but', 'not']
[]
[]
[]
['not']
[]
['but']
[]
[]
['but']
['but', 'not']
[]
[]
[]
['but']
[]
[]
['but', 'but']
['not', 'but']
[]
[]
[]
['but']
[]
[]
['not']
[]
[]
[]
[]
[]
[]
[]
['not', 'nothing']
[]
['not']
[]
['rather']
[]
[]
[]
[]
[]
['not']
[]
['but']
[]
[]
[]
[]
[]
[]
[]
['but']
[]
[]
[]
[]
['not', 'not']
['but', 'never']
[]
[]
[]
[]
['not', 'not']
[]
['not']
['over']
[]
[]
[]
['not']
[]
[]
['but']
['not', 'but']
['not']
[]
['not']
[]
[]
[]
[]
['over']
[]
[]
[]
[]
[]
[]
[]
[]
['but']
[]
['but', 'not']
[]
[]
['not']
[]
[]
[]
[]
[]
['over']
[]
[]
[]
[]
[]
['not']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
["didn't"]
[]
['not']
[]
[]
['but']
['over']
[]
[]
[]
['over']
["didn't"]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['nothing']
[]
[]
['nothing']
['but']
['never', 'but']
[]
["n't"]
[]
[]
[]
[]
[]
[

In [ ]:
string="5 weeks ago"

In [59]:
string.find("week")

TypeError: find/rfind/index/rindex() takes at least 1 argument (0 given)

In [ ]:


if 'month' in string
    print "yes"
    n= re.findall(r'\b\d+\b', string)
    
if week in string

if year i